In [1]:
# The helpers use the application default credentials.
from cmle_training import CMLETraining

execfile('cmle_training.py')

## run locally

### train

In [2]:
import argparse
import tensorflow as tf

In [3]:
def model_fn(features, labels, mode):
    global_step = tf.train.get_or_create_global_step()
    hidden = tf.layers.dense(features, 10, activation=tf.nn.relu)
    outputs = tf.layers.dense(hidden, 1)

    predictions = outputs
    loss = None
    train_op = None

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.nn.l2_loss(outputs - labels)

        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
        train_op = optimizer.minimize(loss, global_step=global_step)

    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op
    )

In [4]:
def train_input_fn():
    fake_features = tf.constant([[1], [2]], dtype=tf.float32)
    fake_labels = tf.constant([[3], [4]], dtype=tf.float32)
    ds = tf.data.Dataset.from_tensor_slices(
        (fake_features, fake_labels)
    )
    return ds.repeat().batch(10)

In [5]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--job-dir',
        default='/tmp/cmle'
    )
    parser.add_argument(
        '--steps',
        type=int,
        default=300)

    args, unused_args = parser.parse_known_args()

    return args

In [6]:
# FIXME: this refers to the other functions, and could be brittle
def train(args):
    estimator = tf.estimator.Estimator(model_fn, model_dir=args.job_dir)
    estimator.train(train_input_fn, steps=args.steps)

    return estimator

In [7]:
# wrapper that handles training locally and on CMLE

toy = CMLETraining(model_fn=model_fn, train_input_fn=train_input_fn, parse_args=parse_args, train=train)

In [8]:
estimator = toy.local_train()

Namespace(job_dir='/tmp/cmle', steps=300)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x108f08b10>, '_model_dir': '/tmp/cmle', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSav

### predict

In [9]:
predict_input_fn = toy.train_input_fn

predictions = estimator.predict(input_fn=predict_input_fn)

In [10]:
print(next(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/cmle/model.ckpt-4500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[3.5]


## run on CMLE

### train

In [11]:
cloud_toy = CMLETraining(job_dir='gs://sandbox-cmle/notebook-cmle/', model_fn=model_fn, train_input_fn=train_input_fn, parse_args=parse_args, train=train)

In [12]:
cloud_toy.cloud_train(project_id='rising-sea-112358', args_dict={'steps': 1000, 'job-dir': cloud_toy.job_dir}, training_input={'runtimeVersion': '1.9'})

Job spec: {'trainingInput': {'runtimeVersion': '1.9', 'scaleTier': 'BASIC', 'region': 'us-central1', 'args': ['--job-dir gs://sandbox-cmle/notebook-cmle/', '--steps 1000'], 'pythonModule': 'trainer.task', 'jobDir': 'gs://sandbox-cmle/notebook-cmle/', 'packageUris': ['gs://sandbox-cmle/notebook-cmle/cmle-sample-0.0.1.tar.gz']}, 'jobId': 'test_1539725654'}


{u'createTime': u'2018-10-16T21:34:15Z',
 u'etag': u'q749/HyXl2I=',
 u'jobId': u'test_1539725654',
 u'state': u'QUEUED',
 u'trainingInput': {u'args': [u'--job-dir gs://sandbox-cmle/notebook-cmle/',
   u'--steps 1000'],
  u'jobDir': u'gs://sandbox-cmle/notebook-cmle/',
  u'packageUris': [u'gs://sandbox-cmle/notebook-cmle/cmle-sample-0.0.1.tar.gz'],
  u'pythonModule': u'trainer.task',
  u'region': u'us-central1',
  u'runtimeVersion': u'1.9'},
 u'trainingOutput': {}}

In [13]:
print(cloud_toy.job_dir)
print(cloud_toy.package_path)

gs://sandbox-cmle/notebook-cmle/
gs://sandbox-cmle/notebook-cmle/cmle-sample-0.0.1.tar.gz


### deploy


### predict